In [ ]:
import csv
import networkx as nx
from functools import lru_cache
from typing import Dict, List, Set, Tuple, Iterable

def natural_sort_key(x):
    """Sort numbers numerically, others lexicographically."""
    try:
        return (0, int(str(x)))
    except ValueError:
        return (1, str(x))

def read_rooted_tree_clusters(
    csv_path: str,
    exclude_trivial: bool = True,
) -> Tuple[Dict[str, Tuple[str, ...]], List[Tuple[str, Tuple[str, ...]]]]:
    """
    Returns:
      clusters_by_node: {node -> sorted tuple of leaf labels under that node}
      clusters_list:    list of (node, cluster) pairs (same info, ordered by node)
    """
    # Build a directed graph from parent->child edges
    G = nx.DiGraph()
    with open(csv_path, newline="") as f:
        reader = csv.reader(f, skipinitialspace=True)
        for row in reader:
            if not row or len(row) < 2:
                continue
            parent, child = row[0].strip(), row[1].strip()
            if parent and child:
                G.add_edge(parent, child)

    if not nx.is_directed_acyclic_graph(G):
        raise ValueError("Input edges do not form a DAG (cycle detected).")

    # Find the (single) root: in-degree 0
    roots = [n for n in G.nodes if G.in_degree(n) == 0]
    if len(roots) != 1:
        raise ValueError(f"Expected exactly 1 root; found {len(roots)}: {roots}")
    root = roots[0]

    # Leaves = nodes with no children
    leaves: Set[str] = {n for n in G.nodes if G.out_degree(n) == 0}

    @lru_cache(maxsize=None)
    def leaves_under(u: str) -> frozenset:
        """Return the set of leaf labels in the subtree rooted at u."""
        if G.out_degree(u) == 0:
            return frozenset([u])  # u itself is a leaf
        acc = set()
        for v in G.successors(u):
            acc |= leaves_under(v)
        return frozenset(acc)

    full_set = leaves_under(root)
    clusters_by_node: Dict[str, Tuple[str, ...]] = {}
    for u in G.nodes:
        cluster = leaves_under(u)
        if exclude_trivial:
            if len(cluster) <= 1:       # singleton
                continue
            if cluster == full_set:      # full set
                continue
        clusters_by_node[u] = tuple(sorted(cluster, key=natural_sort_key))

    # Optional: present as a list of (node, cluster) pairs
    clusters_list: List[Tuple[str, Tuple[str, ...]]] = sorted(
        clusters_by_node.items(), key=lambda kv: natural_sort_key(kv[0])
    )

    return clusters_by_node, clusters_list



: 

In [ ]:
    # Save your sample to a file first, e.g. "rooted.csv", then:
    clusters_by_node, clusters_list = read_rooted_tree_clusters("rooted.csv", exclude_trivial=True)

    print("Clusters by node:")
    for node, cl in clusters_list:
        print(f"{node:>6}: {cl}")
